In [51]:
import numpy as np
import pandas as pd
import re
fileName = "data3.npy" 
data = np.load(fileName, allow_pickle=True)

In [59]:
def tidy_game(data):
    data_string=str(data)
    
    #TAKING GAME OF BUGGED GAMES
    if 'event' not in data_string:
        bugged_Game=["True"]
        bugged_Game=[pd.Series(bugged_Game)]
        df= pd.DataFrame({'bugged_Game': bugged_Game})
        return df
    #EXTRACTING gameID
    result = re.search("', 'gamePk': (.*?), 'link': ", data_string)
    gameID = result.group(1)

    #CUT TO ONLY HAVE ACCESS OF SECTION OF DATA THAT CONTAINS EVENTS
    event_data_string = data_string.partition("'event':")[2]
    
    #SPLIT DATA INTO A LIST OF EVENTS
    events=[]
    events=event_data_string.split("'event': ")

    #KEEP ONLY EVENTS THAT ARE SHOTS OR GOALS
    E=[]
    for x in events:
        if "'Shot'" in x or "'Goal'" in x:
            E.append(x)
    
    #POPULATE GAMEID
    gameID = [gameID]*len(E)

    #POPULATE LIST OF EVENT_TYPE
    event_type=[]
    for x in E:
        if "'Shot'" in x:
            event_type.append('Shot')
        elif "'Goal'" in x:
            event_type.append('Goal')
        else:
            event_type.append(np.nan)

    #POPULATE LIST OF PERIOD
    period=[]
    for x in E:
        if "'Goal'" in x or "'Shot'" in x:
            result = re.search("'period': (.*), 'periodType'", x)
            period.append(result.group(1)) 
        else:
            period.append(np.nan)

    #POPULATE LIST OF PERIOD_TIME
    period_time=[]
    for x in E:
        if "'Goal'" in x or "'Shot'" in x:
            result = re.search("'periodTime': '(.*)', 'periodTimeRemaining'", x)
            period_time.append(result.group(1))
        else:
            period_time.append(np.nan)

    #POPULATE LIST OF PERIOD_TYPE
    period_type=[]
    for x in E:
        if "'Goal'" in x or "'Shot'" in x:
            result = re.search("'periodType': '(.*)', 'ordinalNum'", x)
            period_type.append(result.group(1))
        else:
            period_type.append(np.nan)

    #POPULATE TEAM_OF_SHOOTER
    team_of_shooter=[]
    for x in E:
        if "'Goal'" in x or "'Shot'" in x:
            result = re.search("'team':(.*?)'result'", x)
            if result is None:
                result = re.search("'team':(.*?)'linescore':", x)
            result= re.search("'name': '(.*?)', 'link'", result.group(1))
            team_of_shooter.append(result.group(1))
        else:
            team_of_shooter.append(np.nan)

    #POPULATE X-COORD and Y-COORD
    x_coord=[]
    y_coord=[]
    for x in E:
        if "'Goal'" in x or "'Shot'" in x:
            result1 = re.search(": {'x': (.*?), 'y':", x)
            result2 = re.search(", 'y': (.*?)}, 'team'", x)
            if result1 is None:
                x_coord.append(np.nan)
            else:
                x_coord.append(result1.group(1))
            if result2 is None:
                y_coord.append(np.nan)
            else:    
                y_coord.append(result2.group(1))
        else:
            x_coord.append(np.nan)
            y_coord.append(np.nan)

    #POPULATE SHOOTER
    shooter=[]
    for x in E:    
        if "'Goal'" in x:
            if 'Wrist' in x:
                result = re.search("description': '(.*?) \(\d+\)|- Wrist", x)
                if result is None:
                    result = re.search("description': \"(.*?) \(\d+\)|- Wrist", x)
                shooter.append(result.group(1))         
            if 'Snap' in x:
                result = re.search("description': '(.*?) \(\d+\)|- Snap", x)
                if result is None:
                    result = re.search("description': \"(.*?) \(\d+\)|- Snap", x)    
                shooter.append(result.group(1))
            if 'Slap' in x:
                result = re.search("description': '(.*?) \(\d+\)|- Slap", x)
                if result is None:
                    result = re.search("description': \"(.*?) \(\d+\)|- Slap", x)  
                shooter.append(result.group(1))
            if 'Backhand' in x:
                result = re.search("description': '(.*?) \(\d+\)|- Backhand", x)
                if result is None:
                    result = re.search("description': \"(.*?) \(\d+\)|- Backhand", x)  
                shooter.append(result.group(1))    
            if 'Tip-In' in x:
                result = re.search("description': '(.*?) \(\d+\)|- Tip-In", x)
                if result is None:
                    result = re.search("description': \"(.*?) \(\d+\)|- TipIn", x)  
                shooter.append(result.group(1))
            if 'Wrap-around' in x:
                result = re.search("description': '(.*?) Wrap-around", x)
                if result is None:
                    result = re.search("description': \"(.*?) \(\d+\)|- Wrap-around", x)  
                shooter.append(result.group(1)) 
            if 'Deflected' in x:
                result = re.search("description': '(.*?) Deflected", x)
                if result is None:
                    result = re.search("description': \"(.*?) \(\d+\)|- Deflected", x)  
                shooter.append(result.group(1))  
        
        elif "'Shot'" in x:         
            if 'Wrist' in x:
                result = re.search("description': '(.*?) Wrist", x)
                if result is None:
                    result = re.search("description': \"(.*?) Wrist", x)  
                shooter.append(result.group(1))         
            if 'Snap' in x:
                result = re.search("description': '(.*?) Snap", x)
                if result is None:
                    result = re.search("description': \"(.*?) Snap", x)  
                shooter.append(result.group(1))    
            if 'Slap' in x:
                result = re.search("description': '(.*?) Slap", x)
                if result is None:
                    result = re.search("description': \"(.*?) Slap", x)  
                shooter.append(result.group(1))
            if 'Backhand' in x:
                result = re.search("description': '(.*?) Backhand", x)
                if result is None:
                    result = re.search("description': \"(.*?) Backhand", x)  
                shooter.append(result.group(1))    
            if 'Tip-In' in x:
                result = re.search("description': '(.*?) Tip-In", x)
                if result is None:
                    result = re.search("description': \"(.*?) Tip-In", x)  
                shooter.append(result.group(1))
            if 'Wrap-around' in x:
                result = re.search("description': '(.*?) Wrap-around", x)
                if result is None:
                    result = re.search("description': \"(.*?) Wrap-around", x)  
                shooter.append(result.group(1))
            if 'Deflected' in x:
                result = re.search("description': '(.*?) Deflected", x)
                if result is None:
                    result = re.search("description': \"(.*?) Deflected", x)  
                shooter.append(result.group(1))  
        else:
            shooter.append(np.nan)

    #POPULATE GOALIE
    goalie=[]
    xi=0
    for x in E:
        if "'Goal'" in x:
            goalie.append(np.nan)
        elif "'Shot'" in x: 
            result = re.search("saved by (.*?)',", x)
            if result is None:
                goalie.append(np.nan)
            else:
                goalie.append(result.group(1))
        else:
            goalie.append(np.nan)

        xi+=1
    #POPULATE SHOT_TYPE
    shot_type=[]
    for x in E:
        if 'Wrist' in x:
            shot_type.append('Wrist Shot')         
        elif 'Snap' in x:
            shot_type.append('Snap Shot')   
        elif 'Slap' in x:
            shot_type.append('Slap Shot')
        elif 'Backhand' in x:
            shot_type.append('Backhand')    
        elif 'Tip-In' in x:
            shot_type.append('Tip-In')
        elif 'Wrap-around' in x:
            shot_type.append('Wrap-around')
        elif 'Deflected' in x:
            shot_type.append('Deflected')
        else:
            shot_type.append(np.nan)

    #POPULATE EMPY_NET
    empty_net=[]
    for x in E:
        if "'Shot'" in x:
            empty_net.append(np.nan)
        elif "'Goal'" in x:
            if "REGULAR" in x or "OVERTIME" in x:
                result = re.search("'emptyNet': (.*?)}", x)
                empty_net.append(result.group(1)) 
            else:
                empty_net.append(np.nan)
        else:
            empty_net.append(np.nan)

    #POPULATE STRENGTH
    strength=[]
    for x in E:
        if "'Shot'" in x:
            strength.append(np.nan)
        elif "'Goal'" in x:
            result = re.search("strength'(.*?)},", x)
            result = re.search("'name': '(.*?)'", result.group(1))   
            strength.append(result.group(1))
        else:
            strength.append(np.nan)


    #TRANSFORMING LISTS INTO SERIES
    event_type = pd.Series(event_type)
    period = pd.Series(period)
    period_time = pd.Series(period_time)
    period_type = pd.Series(period_type)
    gameID = pd.Series(gameID)
    team_of_shooter = pd.Series(team_of_shooter)
    x_coord = pd.Series(x_coord)
    y_coord = pd.Series(y_coord)
    shooter = pd.Series(shooter)
    goalie = pd.Series(goalie)
    shot_type = pd.Series(shot_type)
    empty_net = pd.Series(empty_net)
    strength = pd.Series(strength)

    #MAKING THE DATAFRAME WITH THE SERIES
    df = pd.DataFrame({ 'event_type': event_type, 'period': period,'period_time':period_time,'period_type':period_type,'gameID':gameID,
                       'team_of_shooter':team_of_shooter,'x_coord':x_coord,'y_coord':y_coord,'shooter':shooter,'goalie':goalie,
                       'shot_type':shot_type,'empty_net':empty_net,'strength':strength})

    return df

#TESTS EVERYGAME TO SEE IF ANY BREAKS THE FUNCTION:
'''
i=0
for x in range(len(data)):
    if (i%50==0):
        print(i)
    tidy_game(data[x])     
    i+=1
'''
print(":)")


:)


1. In your blog post, include a small snippet of your final dataframe (e.g. using head(10)). You can just include a screenshot rather than fighting to get the tables neatly formatted in HTML/markdown. 

Answer: ok will do

2. You’ll notice that the “strength” field (i.e. even, power play, short handed) only exists for goals, not shots. Furthermore, it doesn’t include the actual strength of players on the ice (i.e. 5 on 4, or 5 on 3, etc). Discuss how you could add the actual strength information (i.e. 5 on 4, etc.) to both shots and goals, given the other event types (beyond just shots and goals) and features available. You don’t need to implement this for this milestone. 

Answer: Let's say there is an event that causes an imbalance of power. In such a cause, the information regarding strenght disparity and the current players per team could be stored, and used to populate the coming events that don't have this information like goal events. 
If an event brings back the strength to even, this would update the strength and players in the ice information so that.

3. In a few sentences, discuss some additional features you could consider creating from the data available in this dataset. We’re not looking for any particular answers, but if you need some inspiration, could a shot or goal be classified as a rebound, or a shot off the rush?

Answer: I noticed that some games have errors and bugs. I would first implement some algorithm that either fixes these games, or simply labels them as "bugged games". I would also attempt to make each event more uniform by making them have the same fields. For example, instead of not having a field "empty net" for 'Shot' events, it could "empty_net: nan". Considering the concept of making new classificaitons for shots and goals, I would go as far as changing all 'Goal' events to be 'Shot' events, and shots that are goals would be a subset of this type of event. 